# Atomic locks using MongoDB

In [ ]:
from pymongo import MongoClient, UpdateOne
from color_helpers import c

client = MongoClient('mongodb://mongodb:27017/')
db = client.atomic_locks_demo

# Initialize the staging area document
db.stagingArea.drop()
db.stagingArea.insert_one({
    "_id": "merchant1",
    "status": "IDLE",
})

Now, we define a simple task to simulate exclusive execution on staging area data:

In [ ]:
def task(process_name, color):
    print(f"{process_name} > {color}{c.BOLD}❚{c.ENDC} Task is being executed...")
    time.sleep(random.uniform(0.1, 0.2))
    print(f"{process_name} > {color}{c.BOLD}❚{c.ENDC} Task completed")
    

## The wrong way: non-atomic lock

In [ ]:
import time
import random

def non_atomic_lock(merchant_id, process_name, task, color, retries=1000):
    for attempt in range(retries):

        staging_area = db.stagingArea.find_one({ "_id": merchant_id })
        if staging_area['status'] == "IDLE":
            
            # Staging Area is IDLE!, Acquire the lock
            print(f"{process_name} > 🔒 Lock acquired")
            time.sleep(random.uniform(0.01, 0.05))  # Simulate a delay to create a race condition
            db.stagingArea.update_one(
              { "_id": merchant_id },
              { "$set": { "status": "PROCESSING" } }
            )

            # Critical Area ########
            task(process_name, color)
            ########################
            
            # Release the lock
            db.stagingArea.update_one(
              { "_id": merchant_id },
              { "$set": { "status": "IDLE" } }
            )
            print(f"{process_name} > 🔓 Lock released")
            return
        else:
            print(f"{c.GRAY}{process_name} > ❌ Failed to acquire lock, current status is {staging_area['status']}. Retrying..{c.ENDC}")
            time.sleep(0.1)
    raise Exception("Timeout")

In [ ]:
from threading import Thread

def simulate_non_atomic_race_condition(iterations):
    merchant_id = "merchant1"
    for _ in range(iterations):
        process1 = Thread(target=non_atomic_lock, args=(merchant_id, "PROCESS1", task, c.GREEN))
        process2 = Thread(target=non_atomic_lock, args=(merchant_id, "PROCESS2", task, c.RED))

        process1.start()
        process2.start()

        process1.join()
        process2.join()


# Run simulations
print("Running non-atomic lock simulation...")
simulate_non_atomic_race_condition(5)


# Implementing atomic locks with MongoDB

In [ ]:
from pymongo.collection import ReturnDocument

def atomic_lock(merchant_id, process_name, task, color, retries=1000):
    for attempt in range(retries):
        # Use findOneAndUpdate to atomically update the status
        result = db.stagingArea.find_one_and_update(
            {"_id": merchant_id, "status": "IDLE"},
            {"$set": { "status": "PROCESSING" }},
            return_document=ReturnDocument.AFTER,
        )

        if not result:
            print(f"{c.GRAY}{process_name} > Failed to acquire lock, current status is not IDLE{c.ENDC}")
            time.sleep(0.1) # Wait for a short time before retrying
            continue

        print(f"{process_name} > 🔒 Lock acquired")
        # Critical Area ########
        task(process_name, color)
        ########################
        
        # Release the lock
        db.stagingArea.update_one(
            {"_id": merchant_id},
            {"$set": {"status": "IDLE"}}
        )
        print(f"{process_name} > 🔓 Lock released")
        return

    raise TimeoutError("All retries exhausted")
            

In [ ]:
def simulate_atomic_race_condition(iterations):
    merchant_id = "merchant1"
    for _ in range(iterations):
        process1 = Thread(target=atomic_lock, args=(merchant_id, "PROCESS1", task, c.GREEN))
        process2 = Thread(target=atomic_lock, args=(merchant_id, "PROCESS2", task, c.RED))

        process1.start()
        process2.start()

        process1.join()
        process2.join()


# Run simulations
print("Running atomic lock simulation...")
simulate_atomic_race_condition(5)